----
----
----


**FrozenBN = BN’nin “dondurulmuş” hali.Yani hem BN’nin normalizasyon davranışına sahiptir,hem de öğrenme/güncelleme özelliklerini kapatır.**

## 🧠 Bu ne demek, pratikte?

Bir koda şöyle bak:
```python
model_bn.train()          # model train modda
bn_layer.forward(x)       # batch istatistiği ile çalışır
```


Ama FrozenBN şöyle davranır:
```python
model_frozen.train()      # model train gibi görünse bile
frozen_bn_layer.forward(x) # running stats + gamma/beta ile çalışır
```


👆 Yani FrozenBN:

* model train() olsa bile BN eval gibi davranır

* running_mean/var hiç güncellenmez

* γ/β çoğu zaman öğrenilmez

* sadece normalize etme hesapsını yapar

### 🧩 Yani FrozenBN “neyi kapatır”?
| Özellik                     | Normal BN | FrozenBN          |
| --------------------------- | --------- | ----------------- |
| Batch istatistiği hesapla   | ✔️        | ❌                 |
| running_mean/var güncelle   | ✔️        | ❌                 |
| γ / β öğren                 | ✔️        | ❌ (çoğu pratikte) |
| normalize et (mean/var ile) | ✔️        | ✔️                |

**✔️ FrozenBN hâlâ normalize eder, sadece öğrenme/güncelleme yapmaz.**

-----
-----
-----


## 🔹 BatchNorm normalde ne yapar?

Train aşamasında

* batch’ten mean/var hesaplar

- running_mean / running_var’ı günceller

Eval / inference aşamasında

* batch’e bakmaz

* running_mean / running_var kullanır

Yani BN zaten iki modlu çalışır.

## 🔹 FrozenBN ne yapıyor?

FrozenBN şunu diyor:

**“Ben bu BN’yi hep eval modundaymış gibi çalıştıracağım.”**

Yani:

❌ Train sırasında batch’ten mean/var alma

❌ running_mean / running_var güncelleme

❌ gamma / beta öğrenme (çoğu kullanımda)

✅ Hep önceden öğrenilmiş istatistiklerle normalize et

-----
-----
-----

## BN / FrozenBN formülü şu: **x̂ = (x - mean) / sqrt(var + eps)**

👉 Yani:

* mean → “merkezle”
* var / std → “ölçekle”

#### Neden sadece mean yetmez?
Şöyle düşün:

* A kümesi: [ -1, 0, 1 ] → dar

* B kümesi: [ -100, 0, 100 ] → çok geniş

**İkisinin de mean’i 0 olabilir ❗**.Ama ölçekleri farklıdır.

* 👉 İşte var, bu farkı yakalar.
* 👉 Var olmadan “her şeyi aynı ölçüye sokamazsın”.

#### 📌 Özet:

* Mean konumu düzeltir,
- Var/Std büyüklüğü düzeltir.

-----
-----
-----


# 4) Normalization Katmanları — Frozen BatchNorm (FrozenBN)

Bu defter, **Frozen BatchNorm** kavramını **CNN / Object Detection / Segmentation** odağında
**sıfırdan ileri seviyeye** anlatır.

Amaç:
- BN’nin *train vs inference* farkını netleştirmek
- FrozenBN’in ne olduğunu ve neden kullanıldığını kavramak
- Pratikte FrozenBN nasıl uygulanır? (kod)
- GN / SyncBN ile karar verme

> Not: Buradaki odak “teoriyi ezberletmek” değil; **doğru sezgi + doğru uygulama**.


---

## 0) FrozenBN için 1 cümle

**Frozen BatchNorm**, BatchNorm katmanının:
- `running_mean / running_var` istatistiklerini **güncellemeyi durdurduğu**
- `gamma/beta` (affine parametrelerini) **öğrenmeyi durdurduğu** (çoğu kullanımda)

ve her zaman **inference davranışı** gibi çalıştığı bir BN türüdür.

Yani: “BN’yi dondur, stabil kalsın.” ❄️


---

## 1) BatchNorm’u doğru anlamadan FrozenBN olmaz

Giriş: `x ∈ R^{N×C×H×W}`

BatchNorm2d, **kanal başına** normalize eder.

### Train modunda (`model.train()`):
- mean/var: **batch içinden** hesaplanır (N ve H×W üzerinden)
- `running_mean` ve `running_var` **EMA ile güncellenir**

### Eval modunda (`model.eval()`):
- mean/var: **running_mean / running_var** kullanılır
- running istatistik **güncellenmez**

Kritik nokta:
> BN’de *train* ve *eval* davranışı **farklıdır**.


---

## 2) FrozenBN neden ortaya çıktı?

Object detection / segmentation eğitiminde sık görülen durum:
- çözünürlük yüksek
- model büyük
- VRAM limiti
- batch genelde küçük: 1–2–4

Batch küçük olunca:
- Train modunda BN’nin batch mean/var’ı **çok gürültülü** olur
- Running stats yanlış yönde “sapabilir”
- Bu da performansı düşürür ve eğitimi kararsız yapar

FrozenBN ile:
- Pretrained backbone’un BN istatistiklerini “sabit kabul edersin”
- Fine-tune sırasında BN “oynamaz”
- Eğitim daha stabil olur

Özet:
> FrozenBN, small-batch rejimde BN’nin kararsızlığını pratik şekilde önler.


---

## 3) FrozenBN tam olarak neyi dondurur?

Pratikte iki farklı “dondurma seviyesi” var:

### Seviye A — Sadece istatistikleri dondur (running stats)
- BN katmanını `eval()` modunda çalıştırırsın
- running_mean/var güncellenmez
- ama gamma/beta (weight/bias) hâlâ öğrenebilir

### Seviye B — İstatistik + affine parametrelerini dondur (en yaygın)
- BN `eval()` gibi çalışır
- ayrıca `weight/bias` için `requires_grad=False`

Detection pipeline’larında genelde **Seviye B** kullanılır.


---

## 4) FrozenBN nerede kullanılır?

Tipik yer:
- **Backbone** (ResNet/ConvNeXt benzeri pretrained ağırlıklar)
- **Neck** bazen
- Head tarafında genelde daha az

En yaygın senaryo:
> “COCO pretrained backbone ile küçük batch fine-tune yapıyorum → FrozenBN”


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

---

## 5) Yaklaşım 1: BN katmanlarını `eval()`’e al (kolay ama eksik)

Bu yöntem:
- running stats güncellemesini durdurur
- ama model `train()` olsa bile sadece BN katmanları eval gibi davranır

İpucu:
- Bu, “istatistikleri dondurma” (Seviye A) sağlar.
- Eğer parametreleri de dondurmak istiyorsan ayrıca `requires_grad=False` yapmalısın.


**Normalize etmek ≠ öğrenmek** 

**BN’yi eval moduna almak, normalization’ı kapatmaz.Sadece normalization için kullanılan mean/variance’ın artık batch’ten değil,önceden öğrenilmiş sabit değerlerden gelmesini sağlar.Yani norm var, öğrenme yok.**

In [2]:
def set_bn_eval(module: nn.Module):
    """Model train moddayken bile BN katmanlarını eval moduna al."""
    for m in module.modules():
        if isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
            m.eval()

m = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True)
)

m.train()
print("Model train() -> BN training flag:", m[1].training)
set_bn_eval(m)
print("BN eval'e zorlandı -> BN training flag:", m[1].training)


Model train() -> BN training flag: True
BN eval'e zorlandı -> BN training flag: False


---

## 6) Yaklaşım 2: FrozenBN modülü (Net ve projelerde taşınabilir)

Bazı projelerde (özellikle detection framework'lerinde) `FrozenBatchNorm2d` diye ayrı bir katman bulunur.
Mantık:
- running stats sabittir
- affine parametreler sabittir
- forward: BN'nin inference formülü

Aşağıdaki implementasyon, konsepti netleştirmek için minimaldir.


In [ ]:
class FrozenBatchNorm2d(nn.Module):
    """Basit FrozenBN implementasyonu."""
    def __init__(self, num_features: int, eps: float = 1e-5):
        super().__init__()
        self.num_features = num_features
        self.eps = eps

        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

        self.weight = nn.Parameter(torch.ones(num_features))
        self.bias = nn.Parameter(torch.zeros(num_features))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mean = self.running_mean.view(1, -1, 1, 1)
        var = self.running_var.view(1, -1, 1, 1)
        w = self.weight.view(1, -1, 1, 1)
        b = self.bias.view(1, -1, 1, 1)
        x_hat = (x - mean) / torch.sqrt(var + self.eps)
        return x_hat * w + b

---

## 7) BN -> FrozenBN dönüşümü (en pratik araç)

Pretrained bir modeldeki `nn.BatchNorm2d` katmanlarını:
- `FrozenBatchNorm2d` ile değiştirebilirsin
- BN’nin running_mean/var + weight/bias değerlerini FrozenBN’e kopyalarsın

Aşağıdaki fonksiyon bunu recursive yapar.


In [ ]:
def convert_bn_to_frozenbn(module: nn.Module) -> nn.Module:
    """nn.BatchNorm2d katmanlarını FrozenBatchNorm2d ile değiştirir (recursive)."""
    for name, child in list(module.named_children()):
        new_child = convert_bn_to_frozenbn(child)

        if isinstance(new_child, nn.BatchNorm2d):
            fb = FrozenBatchNorm2d(new_child.num_features, eps=new_child.eps)
            with torch.no_grad():
                fb.running_mean.copy_(new_child.running_mean)
                fb.running_var.copy_(new_child.running_var)
                if new_child.affine:
                    fb.weight.copy_(new_child.weight)
                    fb.bias.copy_(new_child.bias)
                else:
                    fb.weight.fill_(1.0)
                    fb.bias.zero_()

            # çoğu kullanımda affine dondurulur
            fb.weight.requires_grad_(False)
            fb.bias.requires_grad_(False)

            setattr(module, name, fb)
        else:
            setattr(module, name, new_child)

    return module

In [ ]:
# Demo: BN'li küçük ağ -> FrozenBN'e çevir
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
    nn.Conv2d(16, 16, 3, padding=1, bias=False),
    nn.BatchNorm2d(16),
)

# running stats oluşsun
model_bn.train()
for _ in range(5):
    _ = model_bn(torch.randn(4, 3, 32, 32))

model_frozen = convert_bn_to_frozenbn(model_bn)
print(model_frozen)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): FrozenBatchNorm2d()
  (2): ReLU(inplace=True)
  (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): FrozenBatchNorm2d()
)


---

## 8) FrozenBN gerçekten BN'nin eval davranışını yakalıyor mu?

Kontrol fikri:
- aynı input
- BN modeli `eval()` modunda
- FrozenBN modeli forward
- çıktılar çok yakın olmalı (sayısal farklar küçük olur)

Aşağıdaki test bunu gösterir.


In [ ]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1, bias=False),
    nn.BatchNorm2d(16),
    nn.ReLU(inplace=True),
)

model_bn.train()
for _ in range(10):
    _ = model_bn(torch.randn(4, 3, 32, 32))

model_frozen = convert_bn_to_frozenbn(model_bn)

x = torch.randn(2, 3, 32, 32)

model_bn.eval()
with torch.no_grad():
    y_bn = model_bn(x)
    y_fr = model_frozen(x)

diff = (y_bn - y_fr).abs().mean().item()
print("Mean abs diff (BN eval vs FrozenBN):", diff)

Mean abs diff (BN eval vs FrozenBN): 0.0


---

## 9) FrozenBN ne zaman kötü fikir olabilir?

FrozenBN “her durumda iyi” değildir.

### Kötü olabileceği durumlar
- Domain shift çok büyükse (pretrained istatistikler yeni veriye uymuyorsa)
- Büyük batch ile eğitim yapabiliyorsan (BN zaten sağlıklı çalışır)
- Baştan sona training (scratch) ve batch yeterliyse

Alternatifler:
- **SyncBN** (multi-GPU ile efektif batch büyüt)
- **GN** (batch bağımsız)


---

## 10) FrozenBN, GN ve SyncBN karar ağacı (pratik)

- **Tek GPU / batch küçük (1–4)**:
  - GN çoğu zaman en az sorun çıkarır
  - Pretrained backbone fine-tune: FrozenBN çok yaygın

- **Multi-GPU / Sync altyapısı var**:
  - SyncBN ile BN’yi “gerçek BN” gibi çalıştırabilirsin

- **Pretrained backbone + küçük batch detection**:
  - FrozenBN klasik çözümdür


---

## 11) Kısa özet

- BN train/eval davranışı farklıdır (running stats kritik).
- Batch küçükse BN istatistikleri gürültülü olur → kararsızlık.
- FrozenBN:
  - running stats güncellemesini durdurur
  - çoğu kullanımda affine parametreleri de dondurur
  - pretrained backbone fine-tune için çok pratiktir.

Sonraki konu: **Sync BatchNorm**.
